In [18]:
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
import torch.utils.data as Data
import torch.nn.functional as F
from torchmetrics import R2Score
import utils.dataset as dataset
from models.lstm import ClassificationLSTM
import os

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
# file path
PATH='D:\\Deutschland\\FUB\\master_thesis\\data\\gee\\output'
DATA_DIR = os.path.join(PATH, 'monthly_mean')
LABEL_CSV = 'label.csv'

label_path = os.path.join(PATH, LABEL_CSV)
# files = os.listdir(DATA_DIR)

In [12]:
x_data, y_data = dataset.load_csv_data(DATA_DIR, label_path)

import file D:\Deutschland\FUB\master_thesis\data\gee\output\label.csv
import file D:\Deutschland\FUB\master_thesis\data\gee\output\monthly_mean\1.csv
import file D:\Deutschland\FUB\master_thesis\data\gee\output\monthly_mean\2.csv
import file D:\Deutschland\FUB\master_thesis\data\gee\output\monthly_mean\4.csv
import file D:\Deutschland\FUB\master_thesis\data\gee\output\monthly_mean\5.csv
import file D:\Deutschland\FUB\master_thesis\data\gee\output\monthly_mean\6.csv
import file D:\Deutschland\FUB\master_thesis\data\gee\output\monthly_mean\8.csv
import file D:\Deutschland\FUB\master_thesis\data\gee\output\monthly_mean\9.csv
import file D:\Deutschland\FUB\master_thesis\data\gee\output\monthly_mean\10.csv
import file D:\Deutschland\FUB\master_thesis\data\gee\output\monthly_mean\11.csv
import file D:\Deutschland\FUB\master_thesis\data\gee\output\monthly_mean\12.csv
import file D:\Deutschland\FUB\master_thesis\data\gee\output\monthly_mean\13.csv
import file D:\Deutschland\FUB\master_thesis\

In [13]:
# general hyperparameters
BATCH_SIZE = 128
LR = 0.01
EPOCH = 100
SEED = 2048

1 Classification

1.1 LSTM

In [14]:
# hyperparameters for LSTM
input_size = 5
hidden_size = 20
num_layers = 1
num_classes = 21
layer1_dim = 256
layer2_dim = 128

In [15]:
# embedding
embedding = nn.Embedding(8000, input_size)
# reduce dimention from (n, 1) to (n, )
y_data = y_data.reshape(-1)
x_set = torch.from_numpy(x_data)
y_set = torch.from_numpy(y_data)
x_set = embedding(x_set).detach()

In [ ]:
x_train = x_set[:6347]
x_test = x_set[6347:]
y_train = y_set[:6347]
y_test = y_set[6347:]
train_dataset = Data.TensorDataset(x_train, y_train)
test_dataset = Data.TensorDataset(x_test, y_test)

In [16]:
# build dataset
dataset = Data.TensorDataset(x_set, y_set)
# split dataset
size = len(dataset)
train_size, test_size = round(0.8 * size), round(0.2 * size)
generator = torch.Generator().manual_seed(SEED)
train_dataset, test_dataset = Data.random_split(dataset, [train_size, test_size], generator)
# data_loader
train_loader = Data.DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
test_loader = Data.DataLoader(test_dataset,batch_size=len(test_dataset), shuffle=True,num_workers=2)

In [ ]:
# LSTM model
class simpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, layer1_dim, layer2_dim, num_layers, num_classes):
        super(simpleLSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear1 = nn.Linear(hidden_size, layer1_dim, layer2_dim)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(layer1_dim, layer2_dim)
        self.relu2 = nn.ReLU()
        self.fc = nn.Linear(layer2_dim, num_classes)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        lstm_out, _ = self.lstm(x, (h0, c0))
        tmp1 = self.relu1(self.linear1(lstm_out[:,-1,:]))
        tmp2 = self.relu2(self.linear2(tmp1))
        out = self.fc(tmp2)
        return out

In [19]:
model = ClassificationLSTM(input_size, hidden_size, layer1_dim, layer2_dim, num_layers, num_classes).to(device)
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), LR)

2. Regeression

In [5]:
# hyperparameters for LSTM
input_size = 32
hidden_size = 64
num_layers = 2
num_classes = 9

In [6]:
# embedding
embedding = nn.Embedding(8000, input_size)
# reduce dimention from (n, 1) to (n, )
x_set = torch.from_numpy(x_data)
y_set = torch.from_numpy(y_data).float()
x_set = embedding(x_set).detach()
# build dataset
dataset = Data.TensorDataset(x_set, y_set)
# split dataset
size = len(dataset)
train_size, test_size = round(0.8 * size), round(0.2 * size)
generator = torch.Generator().manual_seed(SEED)
train_dataset, test_dataset = Data.random_split(dataset, [train_size, test_size], generator)
# data_loader
train_loader = Data.DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
test_loader = Data.DataLoader(test_dataset,batch_size=len(test_dataset), shuffle=True,num_workers=2)

In [9]:
# LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Sequential(
                    nn.Linear(hidden_size, 256),
                    nn.ReLU(),
                    nn.BatchNorm1d(256),
                    # 注意 pytorch 的 dropout 的参数和 tf 的参数刚好相反
                    nn.Dropout(0.3),
                    nn.Linear(256, num_classes),
                    nn.Softmax(1)
                )

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        lstm_out, _ = self.lstm(x, (h0, c0))
        out = self.fc(lstm_out[:,-1,:])
        return out

In [10]:
model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)
# loss and optimizer
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), LR)

3. Train and validate model

In [11]:
def train(model:nn.Module, epoch:int):
    total_step = len(train_loader)
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        # forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 50 == 0:
            print('Epoch[{}/{}],Step[{}/{}],Loss:{:.4f}'
            .format(epoch+1,EPOCH,i+50,total_step,loss.item()))

In [12]:
def validate(model:nn.Module):
    model.eval()
    with torch.no_grad():
        for (values, labels) in test_loader:
            values = values.to(device)
            labels = labels.to(device)
            outputs = model(values)
            num = labels.size(0)
            # transpose matrics
            outputs = outputs.t()
            labels = labels.t()
            r2score = R2Score(num_outputs=num, multioutput='raw_values').to(device)
            r2 = r2score(labels, outputs)
        print(f'R^2 on validate set for each class:')
        print('Spruce:{:.2f} | Beech:{:.2f} | Silver fir:{:.2f} | Pine:{:.2f} | Douglas fir:{:.2f} | Oak:{:.2f} | Sycamore:{:.2f} | Coniferous:{:.2f} | Deciduous:{:.2f}'
            .format(r2[0].item(), r2[1].item(), r2[2].item(), r2[3].item(), r2[4].item(), r2[5].item(), r2[6].item(), r2[7].item(), r2[8].item(), r2[9].item()))

In [13]:
for epoch in range(EPOCH):
    train(model, epoch)
    validate(model)

Epoch[1/100],Step[50/50],Loss:0.0296
R^2 on validate set for each class:
Spruce:-9.37 | Beech:-1.09 | Silver fir:-3.15 | Pine:-0.97 | Douglas fir:-9.94 | Oak:-10.67 | Sycamore:-9.75 | Coniferous:-16.30 | Deciduous:-1.12
Epoch[2/100],Step[50/50],Loss:0.0227
R^2 on validate set for each class:
Spruce:-2.33 | Beech:-4.27 | Silver fir:-1.26 | Pine:0.57 | Douglas fir:-2.44 | Oak:-1.04 | Sycamore:-3.90 | Coniferous:-5.45 | Deciduous:-7.00
Epoch[3/100],Step[50/50],Loss:0.0203
R^2 on validate set for each class:
Spruce:0.72 | Beech:-12.79 | Silver fir:0.96 | Pine:-8.64 | Douglas fir:0.58 | Oak:-10.00 | Sycamore:0.52 | Coniferous:-5.72 | Deciduous:0.95
Epoch[4/100],Step[50/50],Loss:0.0177
R^2 on validate set for each class:
Spruce:0.61 | Beech:-27.23 | Silver fir:0.45 | Pine:-17.58 | Douglas fir:-1.06 | Oak:-2.42 | Sycamore:-0.57 | Coniferous:-2.50 | Deciduous:-6.43
Epoch[5/100],Step[50/50],Loss:0.0176
R^2 on validate set for each class:
Spruce:0.59 | Beech:0.76 | Silver fir:-4.94 | Pine:-12.40

KeyboardInterrupt: 

In [ ]:
# save model
torch.save(model, './model.pkl')